In [2]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.multioutput import MultiOutputRegressor
import keras
from keras import layers
from keras.models import Sequential
from keras.layers import Dense, Activation
from keras.callbacks import EarlyStopping
import numpy as np
from keras import metrics
from sklearn.svm import SVR
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.tree import DecisionTreeRegressor

Using TensorFlow backend.


In [3]:
def import_data(path, sep): 
    balance_data = pd.read_csv( path, 
    sep= sep, header = None) 
      
#     Printing the dataset shape 
#     print ("Dataset Lenght: ", len(balance_data)) 
#     print ("Dataset Shape: ", balance_data.shape) 
#     Printing the dataset obseravtions 
#     print ("Dataset: ",balance_data.head()) 
    return balance_data 

In [4]:
def split_dataset(data):
#     distributed data into input parameters X and predicitons Y
#     data = np.asarray(data)
    data = data.loc[1:, :]
    X = data.values[:, :-3]
    Y = data.values[:, -3:]

#     splitting into training and testing datasets
    X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size = 0.1, random_state = 42)
    
    X_train = np.asarray(X_train, dtype=np.float32)
    Y_train = np.asarray(Y_train, dtype=np.float32)
    X_test = np.asarray(X_test, dtype=np.float32)
    Y_test = np.asarray(Y_test, dtype=np.float32)
        
    return X_train, Y_train, X_test, Y_test

In [4]:
path = "./not16nm_Delays_process_temp_pvdd_cqloadNEWLOAD.csv"
path1 = "./AND2_16nm_stat00.csv"
data = import_data(path, ',')
train_x, train_y, test_x, test_y = split_dataset(data)
print(train_x.shape,train_y.shape,test_x.shape,test_y.shape)

/home/ayush/anaconda3/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2903: DtypeWarning: Columns (0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22) have mixed types. Specify dtype option on import or set low_memory=False.
  if self.run_code(code, result):


(45000, 20) (45000, 3) (5000, 20) (5000, 3)


### Neural Network

In [27]:
#import data
#path = "./not16nm_Delays_process_temp_pvdd_cqloadNEWLOAD.csv"
#data = import_data(path, ',')

#data = data.loc[1:, :]
#X = data.values[:, :-3]
#Y = data.values[:, -3:]

#X_train = np.asarray(X, dtype=np.float32)
#Y_train = np.asarray(Y, dtype=np.float32)
    
def NN(X_train,Y_train):
# Initialize the constructor
    model = Sequential()

    # Add an input layer and one hidden
    model.add(Dense(10, activation='tanh', input_shape=(20,), kernel_initializer='normal'))

    model.add(Dense(5, activation='tanh', kernel_initializer='normal'))

    # Add an output layer 
    model.add(Dense(3, kernel_initializer='normal'))

    model.compile(optimizer='adam', loss='mean_squared_error', metrics=['accuracy'])


    # #set early stopping monitor so the model stops training when it won't improve anymore
    # early_stopping_monitor = EarlyStopping(patience=3)

    #train model
    model.fit(X_train, Y_train, validation_split=0.2, epochs=15, verbose=2)
    return { "model" : model }

# results = model.fit(train_x, train_y, epochs=2, batch_size = 500, validation_data = (test_x, test_y))
#print("Test-Accuracy:", np.mean(results.history["val_acc"])))

#test
# test_y_predictions = model.predict(test_x)
# print(test_y_predictions)

# Accuracy
# print("test error: {}%".format(np.mean((test_y_predictions - test_y)**2) * 100))

# scores = model.evaluate(x=test_x, y=test_y, batch_size=None, verbose=2, sample_weight=None, steps=None)

# print('Test loss:', scores)

### Support Vector Regression for Mulitple Outputs

In [23]:
def SVMM(train_x,train_y):
    model1 = MultiOutputRegressor(SVR(kernel='rbf', C=1, gamma=0.1, epsilon=.1))
    predict_Y = model1.fit(train_x, train_y).predict(test_x)
    print(predict_Y.shape,test_y.shape)
#     score = np.square(np.sum(np.subtract(test_y, predict_Y)))*100/5000
    score_1 = mean_squared_error(test_y,predict_Y)
    #scores = model1.evaluate(x=test_x, y=test_y, batch_size=None, verbose=2, sample_weight=None, steps=None)
    #score1 = predict_Y.score(test_x, test_y)
#     print(score_1)
    return { "model" : model1, "mse" : score_1 }
#{ ( 1 / N ) * Σ [ (xi - x) * (yi - y) ] / (σx * σy ) }2
#x = np.subtract(predict_Y, np.mean(predict_Y))
#y = np.subtract(test_y, np.mean(test_y))
#v =np.multiply(np.std(predict_Y),np.std(test_y))
#u = np.multiply(v,5000)
#score2 = np.square(np.sum(np.multiply(x,y))/u)

#print(score2)


### Linear Regression 

In [21]:
def LR(train_x,train_y):
    model2 = MultiOutputRegressor(LinearRegression())
    model2.fit(train_x, train_y)
    predicty = model2.predict(test_x)
    #score2 = model1.score(test_x, test_y)
    score1 = mean_squared_error(predicty, test_y) 
    #score3 = r2_score(predicty, predicty) 
#     score2 = np.square(np.sum(np.subtract(test_y, predicty)))*100/5000
#     print(score1,score2)
    return { "model" : model2, "mse" : score1 } 



### Random Forest Regression

In [31]:
def RF(train_x,train_y,maxdepth1):
    #max_depth1 = 2
    #max_depth2 = 10

    model3 =  RandomForestRegressor(n_estimators=100, max_depth=max_depth1,
                                    random_state=2)
    #model4 =  RandomForestRegressor(n_estimators=100, max_depth=max_depth2,
     #                               random_state=2)

    model3.fit(train_x, train_y)
    #model4.fit(train_x, train_y)

    predict_y3= model3.predict(test_x)
    #predict_y4= model4.predict(test_x)

    #score3_1 = model3.score(test_x, test_y)
    score3_2 = mean_squared_error(predict_y3, test_y)
#     score3_3 = np.square(np.sum(np.subtract(test_y, predict_y3)))*100/5000

    #score4_1 = model4.score(test_x, test_y)
    #score4_2 = mean_squared_error(predict_y4, test_y)
    #score4_3 = np.square(np.sum(np.subtract(test_y, predict_y4)))*100/5000

#     print(score3_2,score3_3)
    return { "model" : model3, "mse" : score3_2 }
    #print(score4_2,score4_3)

### Decision Tree Regression

In [2]:
def DecisionTree(train_x,train_y,maxdepth):    
    #maxdepth= 2
    model5 = DecisionTreeRegressor(max_depth=maxdepth)
    model5.fit(train_x, train_y)

    predict_y5 = model5.predict(test_x)

    #score5_1 = model3.score(test_x, test_y)
    score5_2 = mean_squared_error(predict_y5, test_y)
#     /score5_3 = np.square(np.sum(np.subtract(test_y, predict_y5)))*100/5000

#     print(score5_2, score5_3)
    return { "model" : model5, "mse" : score5_2 }

In [19]:
def Modelret(mod,train_x,train_y):
    if(mod=="SVM"):
        ret = SVMM(train_x,train_y)
    elif(mod=="LR"):
        ret = LR(train_x,train_y)
    elif(mod=="NN"):
        ret = NN(train_x,train_y)
    elif(mod=="RF"):
        ret = RF(train_x,train_y,2)
    elif(mod == "DT"):
        ret = DecisionTree(train_x,train_y,2)
        
        
        
        

In [4]:
model = Modelret("DT",train_x,train_y)

NameError: name 'train_x' is not defined

[[ 1.000e+00  3.900e+01  8.000e-01  1.919e-08  8.091e-10  1.200e-08
   1.268e-09  9.824e+22  1.106e+16  2.619e+25  3.106e+26  2.447e-08
   8.091e-10  1.200e-08  1.268e-09  9.824e+22  2.619e+25  1.106e+16
   3.106e+26  2.447e-08  1.413e-15]
 [ 2.000e+00 -2.800e+01  8.100e-01  2.100e-08  7.722e-10  1.149e-08
   1.383e-09  9.635e+22  1.096e+16  2.759e+25  2.919e+26  2.610e-08
   7.722e-10  1.149e-08  1.383e-09  9.635e+22  2.759e+25  1.096e+16
   2.919e+26  2.610e-08  1.579e-15]
 [ 3.000e+00  1.150e+02  7.400e-01  2.000e-08  8.205e-10  1.217e-08
   1.340e-09  1.017e+23  1.081e+16  2.666e+25  3.132e+26  2.659e-08
   8.205e-10  1.217e-08  1.340e-09  1.017e+23  2.666e+25  1.081e+16
   3.132e+26  2.659e-08  6.130e-16]
 [ 4.000e+00  6.000e+00  8.400e-01  1.905e-08  8.250e-10  1.170e-08
   1.383e-09  1.023e+23  1.078e+16  2.914e+25  3.039e+26  2.680e-08
   8.250e-10  1.170e-08  1.383e-09  1.023e+23  2.914e+25  1.078e+16
   3.039e+26  2.680e-08  2.182e-15]
 [ 5.000e+00  6.600e+01  7.200e-01  1.98

In [8]:
def run(model, gate):
    if gate == "AND3":
        data = import_data('./AND3/AND_3_delay_16nm.csv', ',')
        data.head()
        data = data.values[1:, :]
        # print(data[:5, :])
        X = data[:, :-6]
        Y = data[:, -6:]
        X_del = np.asarray(X, dtype=np.float32)
        Y_del = np.asarray(Y, dtype=np.float32)
#         print(X[:5, :])
#         print(Y[:5, :])
        model_and3 = Modelret(model, X_del, Y_del)